# Create Heightmap from source DEM

## Setting up the Environment

Before you can run the notebook, one must first create the environment with the necessary dependencies.

If you don't have `Git`, download it selecting the appropriate OS on your machine: https://git-scm.com/downloads. If you are using Windows, click on the `Click here to download` link to download the most up to date version.

As well, if you don't have `Anaconda`, download it following this link: https://www.anaconda.com/download/success.

Having `Git` and `Anaconda` downloaded follow the steps outlined:

- Open a command line terminal and change the working directory to where you want your height map outputted.
- Run the following command: `git clone https://github.com/fakurten94/Heightmap-Generation`
- Having the `Heightmap-Generation` folder downloaded, go to the command line (if using Windows switch to an Anaconda prompt) and run `cd Heightmap-Generation`
- Run `conda env create -n name_of_environment_you_want --file environment.yml`. This step may take up to an hour to complete.
- Once the environment is created, in the terminal (or Anaconda Prompt if in Windows) run `conda activate name_of_environment_you_created`
- Run `pip install localtileserver`
- Finally run `jupyter notebook`
- If a web server was not open using the last command or gave you an error in the terminal run `pip install jupyter notebook`.

After following all steps, you may proceed to the next section.

## Importing the Python Libraries

In [ ]:
import copy
import geopandas as gpd
import ipyleaflet
import ipywidgets as widgets
import json
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
from osgeo import gdal
import pdal
import pyproj
import requests
from shapely.geometry import shape, Point, Polygon
from shapely.ops import transform
import folium
import rasterio
from localtileserver import TileClient, get_leaflet_tile_layer
from localtileserver import examples, helpers
from ipyleaflet import Map, SplitMapControl

## Converting a ROI of a DEM to a 16-bit Grayscale PNG

This notebook was designed for a user to define the region of interest from a DEM and be able to export it into a 16 bit png. 

Please run all the following code cells in this notebook.

The following code cell is defining some of the functions that are going to be used in the notebook.

In [ ]:
def handle_draw(target, action, geo_json):
    user_ROI.append(geo_json)

def heightmap_generation(roi):
    aoi_bounds = roi[0]["geometry"]["coordinates"][0][:-1]
    top_left,bottom_right = aoi_bounds[1],aoi_bounds[3]
    tl_crs = transformer_latlong_crs.transform(top_left[1],top_left[0])
    br_crs = transformer_latlong_crs.transform(bottom_right[1],bottom_right[0])
    
    window = (tl_crs[0],tl_crs[1],br_crs[0],br_crs[1])
    
    options = gdal.TranslateOptions(format='GTiff', projWin=window)
    gdal.Translate(f"{output_file.split('.')[0]}_temp.tif",input_file,options=options)
    
    with rasterio.open(f"{output_file.split('.')[0]}_temp.tif") as f:
        arr = f.read(1)
        min_val, max_val = int(np.min(arr)), int(np.max(arr))
    
    cli = f"gdal_translate -co WORLDFILE=YES -ot UInt16 -scale {min_val} {max_val} 0 65536 {output_file.split('.')[0]}_temp.tif {output_file}"
    os.system(cli)

def fxn():
    warnings.warn("No overview",NoOverviewWarning)

def display_dem(input_file):
    user_AOI = []
    client = TileClient(input_file)
    layer = get_leaflet_tile_layer(client,name=input_file.split(".")[0])
    
    m = ipyleaflet.Map(name=input_file,center=client.center(), zoom=client.default_zoom)
    m.add(layer)
    
    control = ipyleaflet.LayersControl(position='topright')
    dc = ipyleaflet.DrawControl(
        polygon={},
        rectangle={"pathOptions": {"color": "blue"}},
        circlemarker={},
        polyline={}
    )
    dc.on_draw(handle_draw)
    m.add(control)
    m.add_control(dc)
    
    print("Please select the ROI using the Rectangle command on the left side of the map.")
    display(m)
    warnings.filterwarnings('ignore')


def display_raster(output_file):
    client_ = TileClient(output_file)
    tdem = get_leaflet_tile_layer(client_, colormap='gray', nodata=0)
    dem = client_.dataset.read()[0, :, :]
    
    hs_arr = helpers.hillshade(dem)
    
    hs = rasterio.open(helpers.save_new_raster(client_, hs_arr))
    hst = get_leaflet_tile_layer(hs, nodata=0)
    
    d = client_.get_leaflet_map()
    control = SplitMapControl(left_layer=tdem, right_layer=hst)
    d.add_control(control)
    display(d)

def colorbar_scale(output_file):
    with rasterio.open(f"{output_file.split('.')[0]}_temp.tif") as src:
        arr = src.read(1)
        min_val, max_val = int(np.min(arr)), int(np.max(arr))
        print(f"The minimum value is {min_val} and the maximum value is {max_val}")
        plt.figure(figsize=(8,8))
        plt.imshow(src.read(1), cmap='gray');
        x = plt.colorbar();
        x.set_ticks([min_val,max_val])
        ax = plt.gca()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.show();

In this next cell, please provide the names of the input and output files. On the output file type the `.png` extension.

In [ ]:
input_file = input("Please provide the input file name and location (Ex:'data/DEM.tif')")
output_file = input("Please provide the output file name and location (Ex:'data/DEM.png')")

In [ ]:
with rasterio.open(input_file) as dataset:
    crs = str(dataset.crs)

transformer_latlong_crs = pyproj.Transformer.from_crs("EPSG:4326",crs)
transformer_crs_latlong = pyproj.Transformer.from_crs(crs,"EPSG:4326")

The next code cell will display the input DEM provided and a ROI can be selected using the commands on the left side of the map. This may take a while depending on the size of the DEM.

In [ ]:
user_ROI = []
display_dem(input_file)

The following cell will grab the ROI selected above and will generate the 16 bit PNG file.

In [ ]:
assert user_ROI != [], "Please select a ROI before running this code cell."
heightmap_generation(user_ROI)

The generated file will be displayed here in this map along with the corresponding hillshade. One will be able to compare both by dragging the control to the left or right. 

In [ ]:
display_raster(output_file)

If the display of the above cell is somewhere off the coast of Africa, rerun the notebook **after** following the outlined step below.

- If on Windows run on the Osgeo Shell `set GDAL_PAM_ENABLED=YES`
- If on Linux or Mac run on the terminal `export GDAL_PAM_ENABLED=YES`

The next cell will display the min and max elevations in the selected ROI.

In [ ]:
colorbar_scale(output_file)

Run the next cell to verify that the selected ROI does not have no no data values. If the output displays `There are X no data values in output file` please reselect the ROI from the beginning. **Important: Restart the kernel if attempting to reselect a different ROI.**

In [ ]:
with rasterio.open(output_file) as f:
    arr = f.read(1)
    mask = (arr != f.nodata)
    elev = arr[mask]
    if arr[arr==f.nodata].shape[0] == 0: print("No no data values found in the output file.")
    else: print(f'There are {arr[arr==f.nodata].shape[0]} no data values in the output file. Please reselect ROI.')